# Chapter 07-05: 생성 모델 — GAN (Generative Adversarial Network)

## 학습 목표
- GAN의 목적 함수와 생성자/판별자의 역할을 이해한다
- DCGAN(Deep Convolutional GAN)의 Generator와 Discriminator를 구현한다
- 커스텀 학습 루프로 생성자와 판별자를 교대 업데이트한다
- 에포크별 생성 이미지를 시각화하여 학습 진행을 모니터링한다
- GAN 학습 불안정성의 원인과 해결책을 이해한다

## 목차
1. GAN 목적 함수
2. DCGAN Generator 구현
3. DCGAN Discriminator 구현
4. 커스텀 GAN 학습 루프
5. 에포크별 생성 이미지 시각화
6. 학습 불안정성과 해결책
7. 정리

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib
import time

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

print(f'TensorFlow 버전: {tf.__version__}')

# 재현성 시드 고정
tf.random.set_seed(42)
np.random.seed(42)

## 1. GAN 목적 함수

GAN은 **생성자(Generator, G)**와 **판별자(Discriminator, D)**가 서로 경쟁하는 미니맥스 게임:

$$\min_G \max_D\; \mathbb{E}[\log D(x)] + \mathbb{E}[\log(1-D(G(z)))]$$

| 항 | 해석 | 최적화 방향 |
|----|------|------------|
| $\mathbb{E}[\log D(x)]$ | 실제 이미지 $x$를 실제로 판별할 확률 | D는 **최대화** (1에 가깝게) |
| $\mathbb{E}[\log(1-D(G(z)))]$ | 가짜 이미지 $G(z)$를 가짜로 판별할 확률 | D는 **최대화**, G는 **최소화** |

### 직관적 이해

```
Generator G: 노이즈 z → 가짜 이미지 G(z) → D를 속이려 한다
Discriminator D: 실제/가짜 이미지 → 진위 확률 D(x) → G를 검출하려 한다
```

- **균형점(Nash Equilibrium)**: D(x) = 0.5 → 판별자가 더 이상 구분 불가
- **실제 학습**: G 손실 = $-\log(D(G(z)))$ (Non-saturating 변형으로 기울기 안정화)

### 손실 함수 (이진 교차 엔트로피 기반)

- **D 손실**: $-\mathbb{E}[\log D(x)] - \mathbb{E}[\log(1-D(G(z)))]$
  - 실제 이미지 레이블 = 1, 가짜 이미지 레이블 = 0
- **G 손실**: $-\mathbb{E}[\log D(G(z))]$
  - 가짜 이미지를 실제(레이블=1)로 속이려 함

## 2. DCGAN Generator 구현

In [ ]:
LATENT_DIM = 100  # 잠재 벡터(노이즈) 차원

def build_generator(latent_dim=LATENT_DIM):
    """
    DCGAN Generator
    노이즈 z (latent_dim,) → 이미지 (28, 28, 1)
    
    구조: Dense → Reshape → Conv2DTranspose 스택 → tanh 출력
    """
    model = keras.Sequential(name='Generator')
    
    # --- Dense + Reshape ---
    # 잠재 벡터를 작은 공간 맵으로 변환 (7×7×256 특징 맵)
    model.add(keras.layers.Dense(7 * 7 * 256, use_bias=False,
                                  input_shape=(latent_dim,)))
    model.add(keras.layers.BatchNormalization())   # 배치 정규화로 학습 안정화
    model.add(keras.layers.LeakyReLU(0.2))        # LeakyReLU (음수 기울기 허용)
    
    model.add(keras.layers.Reshape((7, 7, 256)))   # (7, 7, 256)
    
    # --- Conv2DTranspose (전치 합성곱) 스택 ---
    # 공간 해상도를 점진적으로 2배씩 업샘플링
    
    # (7, 7, 256) → (7, 7, 128)
    model.add(keras.layers.Conv2DTranspose(
        128, kernel_size=5, strides=1, padding='same', use_bias=False
    ))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU(0.2))
    
    # (7, 7, 128) → (14, 14, 64)  ← strides=2로 해상도 2배
    model.add(keras.layers.Conv2DTranspose(
        64, kernel_size=5, strides=2, padding='same', use_bias=False
    ))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU(0.2))
    
    # (14, 14, 64) → (28, 28, 1)  ← 최종 출력층
    model.add(keras.layers.Conv2DTranspose(
        1, kernel_size=5, strides=2, padding='same',
        use_bias=False, activation='tanh'  # tanh: 출력 범위 [-1, 1]
    ))
    
    return model


generator = build_generator()
generator.summary()

# 동작 확인
test_noise = tf.random.normal([1, LATENT_DIM])
test_gen_output = generator(test_noise, training=False)
print(f'\n노이즈 shape: {test_noise.shape}')
print(f'생성 이미지 shape: {test_gen_output.shape}')  # (1, 28, 28, 1)
print(f'픽셀 범위: [{test_gen_output.numpy().min():.3f}, {test_gen_output.numpy().max():.3f}]')

## 3. DCGAN Discriminator 구현

In [ ]:
def build_discriminator():
    """
    DCGAN Discriminator
    이미지 (28, 28, 1) → 확률 스칼라 (0~1)
    
    구조: Conv2D 스택 → Flatten → Dense → Sigmoid 출력
    배치 정규화 대신 Dropout 사용 (GAN 판별자 권장)
    """
    model = keras.Sequential(name='Discriminator')
    
    # --- Conv2D 스택 (해상도 점진적 축소) ---
    
    # (28, 28, 1) → (14, 14, 64)
    model.add(keras.layers.Conv2D(
        64, kernel_size=5, strides=2, padding='same',
        input_shape=(28, 28, 1)
    ))
    model.add(keras.layers.LeakyReLU(0.2))  # 음수 기울기 허용 (0.2 slope)
    model.add(keras.layers.Dropout(0.3))    # 과적합 방지 (BN 대신 Dropout 사용)
    
    # (14, 14, 64) → (7, 7, 128)
    model.add(keras.layers.Conv2D(
        128, kernel_size=5, strides=2, padding='same'
    ))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.3))
    
    # --- 분류 헤드 ---
    model.add(keras.layers.Flatten())       # (7*7*128,) = (6272,)
    model.add(keras.layers.Dense(
        1, activation='sigmoid'             # 0(가짜) ~ 1(실제) 확률
    ))
    
    return model


discriminator = build_discriminator()
discriminator.summary()

# 동작 확인
test_image = tf.random.normal([1, 28, 28, 1])
test_d_output = discriminator(test_image, training=False)
print(f'\n입력 이미지 shape: {test_image.shape}')
print(f'판별 결과 shape: {test_d_output.shape}')      # (1, 1)
print(f'판별 확률 (초기): {test_d_output.numpy()[0][0]:.4f}')  # 초기에는 무작위

## 4. 커스텀 GAN 학습 루프

In [ ]:
# MNIST 데이터 준비
(x_train, _), (_, _) = keras.datasets.mnist.load_data()

# 정규화: [0, 255] → [-1, 1] (tanh 출력과 일치)
x_train = x_train.astype('float32')
x_train = (x_train - 127.5) / 127.5        # [-1, 1] 범위로 정규화
x_train = x_train[..., np.newaxis]          # (60000, 28, 28) → (60000, 28, 28, 1)

print(f'훈련 데이터 shape: {x_train.shape}')
print(f'픽셀 범위: [{x_train.min():.1f}, {x_train.max():.1f}]')

# TF Dataset 파이프라인
BUFFER_SIZE = 60000
BATCH_SIZE  = 256

train_dataset = (
    tf.data.Dataset.from_tensor_slices(x_train)
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.AUTOTUNE)  # 비동기 데이터 로딩
)

# 손실 함수: 이진 교차 엔트로피 (from_logits=False, 이미 sigmoid 적용됨)
bce_loss = keras.losses.BinaryCrossentropy(from_logits=False)

def discriminator_loss(real_output, fake_output):
    """
    판별자 손실:
    - 실제 이미지 → 1로 분류 (real_loss)
    - 가짜 이미지 → 0으로 분류 (fake_loss)
    """
    real_loss = bce_loss(tf.ones_like(real_output), real_output)   # 실제=1
    fake_loss = bce_loss(tf.zeros_like(fake_output), fake_output)  # 가짜=0
    return real_loss + fake_loss

def generator_loss(fake_output):
    """
    생성자 손실 (Non-saturating):
    가짜 이미지를 실제(1)로 속이도록 학습
    """
    return bce_loss(tf.ones_like(fake_output), fake_output)        # 가짜를 1로

# 별도 Optimizer (D와 G가 독립적으로 학습)
gen_optimizer  = keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)  # GAN 권장
disc_optimizer = keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5)

print('손실 함수 및 옵티마이저 설정 완료')

@tf.function  # 그래프 모드로 컴파일 → 속도 향상
def train_step(real_images):
    """
    한 배치의 GAN 학습 스텝:
    1. G: 노이즈 → 가짜 이미지 생성
    2. D: 실제/가짜 이미지 판별 후 손실 계산 → D 업데이트
    3. G: 판별 결과로 손실 계산 → G 업데이트
    """
    batch_size = tf.shape(real_images)[0]
    
    # 노이즈 샘플링 (z ~ N(0, I))
    noise = tf.random.normal([batch_size, LATENT_DIM])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # G 순전파: 가짜 이미지 생성
        fake_images = generator(noise, training=True)
        
        # D 순전파: 실제/가짜 이미지 판별
        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(fake_images, training=True)
        
        # 손실 계산
        gen_loss  = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    # D 역전파 (D의 파라미터만 업데이트)
    disc_grads = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    disc_optimizer.apply_gradients(zip(disc_grads, discriminator.trainable_variables))
    
    # G 역전파 (G의 파라미터만 업데이트)
    gen_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gen_optimizer.apply_gradients(zip(gen_grads, generator.trainable_variables))
    
    return gen_loss, disc_loss


# 학습 루프
EPOCHS = 20

# 시각화용 고정 노이즈 (학습 진행 모니터링)
seed_noise = tf.random.normal([16, LATENT_DIM])

gen_losses  = []
disc_losses = []
saved_images = []  # 에포크별 생성 이미지 저장

print(f'GAN 학습 시작 (총 {EPOCHS} 에포크)...')
start_time = time.time()

for epoch in range(EPOCHS):
    epoch_gen_loss  = []
    epoch_disc_loss = []
    
    for batch in train_dataset:
        g_loss, d_loss = train_step(batch)
        epoch_gen_loss.append(g_loss.numpy())
        epoch_disc_loss.append(d_loss.numpy())
    
    avg_g_loss = np.mean(epoch_gen_loss)
    avg_d_loss = np.mean(epoch_disc_loss)
    gen_losses.append(avg_g_loss)
    disc_losses.append(avg_d_loss)
    
    # 에포크별 생성 이미지 저장
    generated = generator(seed_noise, training=False).numpy()
    saved_images.append(generated)
    
    elapsed = time.time() - start_time
    print(f'에포크 {epoch+1:3d}/{EPOCHS} | '
          f'G 손실: {avg_g_loss:.4f} | '
          f'D 손실: {avg_d_loss:.4f} | '
          f'경과: {elapsed:.1f}초')

print('\n학습 완료!')

## 5. 에포크별 생성 이미지 격자 시각화

In [ ]:
def plot_generated_images(images, epoch, n_rows=4, n_cols=4):
    """
    생성 이미지를 n_rows × n_cols 격자로 시각화
    images: shape = (n, 28, 28, 1), 픽셀 범위 [-1, 1]
    """
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 10))
    
    for i, ax in enumerate(axes.flat):
        if i < len(images):
            # [-1, 1] → [0, 1]로 변환
            img = (images[i, :, :, 0] + 1) / 2.0
            ax.imshow(img, cmap='gray')
        ax.axis('off')
    
    plt.suptitle(f'에포크 {epoch}: GAN 생성 이미지', fontsize=13, fontweight='bold')
    plt.tight_layout()
    plt.show()


# 초기/중간/최종 에포크 이미지 비교
milestone_epochs = [1, EPOCHS//4, EPOCHS//2, EPOCHS]

for ep in milestone_epochs:
    idx = min(ep - 1, len(saved_images) - 1)
    plot_generated_images(saved_images[idx], epoch=ep)

# 손실 곡선
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(range(1, EPOCHS+1), gen_losses,  label='Generator 손실',     linewidth=2, color='blue')
ax.plot(range(1, EPOCHS+1), disc_losses, label='Discriminator 손실', linewidth=2, color='red')
ax.axhline(y=np.log(2), color='gray', linestyle='--', alpha=0.7, label='이론적 균형점 (ln2 ≈ 0.693)')
ax.set_xlabel('에포크', fontsize=11)
ax.set_ylabel('손실', fontsize=11)
ax.set_title('GAN 학습 손실 곡선', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f'최종 Generator 손실:     {gen_losses[-1]:.4f}')
print(f'최종 Discriminator 손실: {disc_losses[-1]:.4f}')
print(f'이론적 균형점:            {np.log(2):.4f} (ln 2)')

## 6. 학습 불안정성 원인과 해결책

### 주요 문제 1: Mode Collapse (모드 붕괴)

**현상**: Generator가 항상 동일하거나 매우 유사한 이미지만 생성  
**원인**: D를 쉽게 속이는 특정 모드에만 집중하는 지름길 학습

```
예: MNIST에서 항상 숫자 '1'만 생성하여 D를 속이는 경우
```

**해결책**:
- **Minibatch Discrimination**: 배치 내 다양성을 D 입력에 추가
- **Feature Matching**: G가 D의 중간 레이어 특징 분포를 매칭
- **Wasserstein GAN (WGAN)**: Earth Mover Distance로 손실 대체
- **Mode Seeking Regularization**: 다양한 z → 다양한 이미지 장려

### 주요 문제 2: Vanishing Gradient (기울기 소실)

**현상**: D가 너무 잘 학습되면 G의 기울기가 0에 가까워져 G 학습 불가  
**원인**: 완벽한 D → $D(G(z)) \approx 0$ → $\log(1-D(G(z))) \approx 0$의 기울기가 소실

**해결책**:
- **Non-Saturating Loss**: $\log(D(G(z)))$ 최대화로 변경 (기울기 포화 방지)
- **Label Smoothing**: 실제 이미지 레이블을 1 대신 0.9로 설정
- **WGAN-GP**: Gradient Penalty로 D의 Lipschitz 제약 강화

### 주요 문제 3: 학습 불안정 (Oscillation)

**현상**: G와 D 손실이 수렴하지 않고 진동  
**해결책**:
- **Two Time-Scale Update Rule (TTUR)**: G와 D에 다른 학습률 적용
- **Spectral Normalization**: D의 가중치 정규화로 안정화
- **D 업데이트 빈도 조절**: G 1회 업데이트 당 D를 n회 업데이트

### GAN 변형 모델 비교

| 모델 | 핵심 아이디어 | 장점 |
|------|-------------|------|
| **DCGAN** | Conv2DTranspose 기반 | 이미지 생성 기준 모델 |
| **WGAN** | Wasserstein Distance | Mode Collapse 감소 |
| **WGAN-GP** | Gradient Penalty | 학습 안정화 |
| **StyleGAN** | 스타일 제어 | 고화질 얼굴 생성 |
| **CycleGAN** | 비쌍 이미지 변환 | 도메인 변환 |
| **Pix2Pix** | 쌍 이미지 변환 | 조건부 이미지 생성 |

## 7. 정리

### DCGAN 구현 핵심 요약

| 요소 | Generator | Discriminator |
|------|-----------|---------------|
| 입력 | 노이즈 $z \sim \mathcal{N}(0,I)$ | 이미지 (28×28×1) |
| 출력 | 이미지 (28×28×1) | 확률 스칼라 [0,1] |
| 핵심 레이어 | Conv2DTranspose | Conv2D |
| 정규화 | BatchNormalization | Dropout |
| 활성화 | LeakyReLU + tanh(출력) | LeakyReLU + sigmoid(출력) |
| 손실 목표 | D(G(z)) → 1 | D(x) → 1, D(G(z)) → 0 |

### 학습 팁

- **학습률**: G와 D 모두 `2e-4`, `beta_1=0.5` (Adam)
- **픽셀 정규화**: 입력 이미지를 `[-1, 1]`로 (tanh 출력과 일치)
- **배치 크기**: 64 ~ 256 (크면 학습 안정적)
- **D 먼저 업데이트**: 일반적으로 D → G 순서로 업데이트
- **균형 모니터링**: G 손실 ≈ D 손실 ≈ `ln(2) ≈ 0.693` 이 이상적